## Segmentação dos dados

As características normalmente são extraídas sobre pequenos segmentos de tamanho fixo dos dados, não no dado como um todo. Estes pequenos segmentos são chamados de janelas. A técnica de separar os dados em janelas, recebe o nome de janela deslizante (*sliding window*) e é uma forma de segmentação de dados. Uma boa prática, é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações da lacuna de uma janela e outra não sejam perdidas. Na imagem a seguir podemos observar um sinal EOG. Abaixo dele a representação de janelas: *W1*, *W2*, *W3*... Repare que entre as janelas, há uma sobreposição de tamanho *T*. 

Na base de dados que iremos implementar, cada ensaio tem duração de 5 segundos. Se utilizadas janelas com tamanho de 250 ms, resultará em 20 janelas. Ao aplicar uma sobreposição de ~128ms, ficamos com 41 janelas de ~122ms.

Algumas características de EMG trabalham com o dado no domínio da frequência. Quando tais características são aplicadas, é necessário tranformar o dado para o domínio da frequência, utilizando o método de transformação de domínio `STFT`, do inglês *Short-time Fourier transform* (Transformada de Fourier de curto termo). O código divide o dado em segmentos, tanto no domínio do tempo quanto no domínio da frequência.

In [4]:
# Criando dados fictícios

import numpy as np

shape = (60, 4, 1000)
data = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)
data.shape, data

((28, 1000, 4),
 array([[[-196.79167175,  -52.67275238,    0.        ,    0.        ],
         [-220.41287231,  -62.84153748,    0.        ,    0.        ],
         [-214.33554077,  -57.58324051,    0.        ,    0.        ],
         ...,
         [-197.27784729, -356.38815308,    0.        ,    0.        ],
         [-192.95826721, -354.00958252,    0.        ,    0.        ],
         [-189.22210693, -344.97024536,    0.        ,    0.        ]],
 
        [[-158.54745483,  -55.20092392,    0.        ,    0.        ],
         [-146.16090393,  -48.95154953,    0.        ,    0.        ],
         [-165.29797363,  -54.07895279,    0.        ,    0.        ],
         ...,
         [-249.46253967,   57.25973892,    0.        ,    0.        ],
         [-238.3961792 ,   59.35782242,    0.        ,    0.        ],
         [-254.63856506,   54.95970154,    0.        ,    0.        ]],
 
        [[-168.12159729,   -9.57040405,    0.        ,    0.        ],
         [-176.80938721,  -

In [6]:
from scipy.signal import stft

step = 29
segment = 64
print('', data.shape)

n_win = int((data.shape[1] - segment) / step) + 1  # Adjusted for the second dimension
ids = np.arange(n_win) * step

# janelas do dado no domínio do tempo
chunks_time = np.array([data[:, k:(k + segment), :].transpose(0, 2, 1) for k in ids])

# janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=segment, noverlap=step-1)  # Adjusted noverlap
chunks_freq = chunks_freq.transpose(0, 2, 3, 1)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Domínio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Domínio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

 (28, 1000, 4)


/home/moraski/.local/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 24 is greater than input length  = 4, using nperseg = 4
  warnings.warn('nperseg = {0:d} is greater than input length '


ValueError: noverlap must be less than nperseg.

*Desafio*: Implementar um algoritmo para encontrar a mesma quantidade de janelas em ambos os domínios (tempo e frequência).